<a href="https://colab.research.google.com/github/PTC-Education/PTC-API-Playground/blob/main/Onshape_Research_Symposium_2022_Countdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Onshape Countdown Timer
This python notebook is set up to control [this public Onshape document](https://cad.onshape.com/documents/ed30f64021f3697e6f786867/w/2927a853e746a5a7e0e57cd9/e/c2de207d4ed142dbfd08feba) to count down a specified duration of time.

For more info on the Onshape REST API, you can visit [the developer portal](https://onshape-public.github.io/docs/). A library of example python notebooks and getting started tutorials [can be found on this page](https://ptc-education.github.io/docs/solutions/onshapedx).


In [1]:
#@title Import and Setup Onshape Client

!pip install onshape-client
from onshape_client.client import Client
from onshape_client.onshape_url import OnshapeElement
import json

#@markdown Chage the base if using an enterprise (i.e. "https://ptc.onshape.com")
base = 'https://cad.onshape.com' #@param {type:"string"}

#@markdown Would you like to import your API keys from a file, or copy and paste them directly?
keyImportOption = "Copy/Paste Keys" #@param ["Upload Keys from File", "Copy/Paste Keys"]

from IPython.display import clear_output 
clear_output()
print("Onshape Client successfully imported!")

if keyImportOption == "Upload Keys from File":
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    execfile(fn)

  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')
else:
  access = input("Paste your Onshape Access Key: ")
  secret = input("Paste your Onshape Secret Key: ")
  client = Client(configuration={"base_url": base,
                                "access_key": access,
                                "secret_key": secret})
  clear_output()
  print('Onshape client configured - ready to go!')


Onshape client configured - ready to go!


### Define funcitons for getting and setting variables

In [2]:
#@title Get Variables from Variable Studio or Part Studio
#@markdown Defines function `getVariables(url)`, which returns JSON of all variables in a variable studio or part studio
url = 'https://cad.onshape.com/documents/ed30f64021f3697e6f786867/w/2927a853e746a5a7e0e57cd9/e/9df078c75304fa3b3642b9a4' #@param {type:"string"}
showResponse = True #@param {type:"boolean"}
#@markdown **Also defines** function `getVariableValue(url,config,varName)`

#@markdown Input a variable name to get value for that variable.
varName = 'timeRemaining' #@param {type:"string"}


def getVariables(url: str):
  fixed_url = '/api/variables/d/did/w/wid/e/eid/variables'
  element = OnshapeElement(url)
  method = 'GET'

  params = {}
  payload = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

  parsed = json.loads(response.data)
  return parsed
if showResponse:
  vars = getVariables(url)
  print(json.dumps(vars, indent=4, sort_keys=True))
else:
  pass

def getVariableValue(url,varName):
  vars = getVariables(url)
  for x in vars[0]['variables']:
    if varName in x['name']:
      return(x['expression'])

if varName is not "":
  value = getVariableValue(url,varName)
  print(varName + " is equal to " + value)

[
    {
        "variableStudioReference": null,
        "variables": [
            {
                "description": "",
                "expression": "\"Right now!\"",
                "name": "timeRemaining",
                "type": "ANY",
                "value": null
            },
            {
                "description": "",
                "expression": "3.4125*cm",
                "name": "hourGlassTime",
                "type": "LENGTH",
                "value": null
            }
        ]
    }
]
timeRemaining is equal to "Right now!"


In [3]:
#@title Set Variable in Variable Studio
#@markdown Defines function `setVariable(url, varName, varValue)`, which updates the value of specified name with specified variable
url = 'https://cad.onshape.com/documents/ed30f64021f3697e6f786867/w/2927a853e746a5a7e0e57cd9/e/9df078c75304fa3b3642b9a4' #@param {type:"string"}
varName = 'timeRemaining' #@param {type:"string"}
varValue = "\"hello\"" #@param {type:"string"}
showResponse = True #@param {type:"boolean"}

def setVariable(url: str, varName: str, varValue):
  fixed_url = '/api/variables/d/did/w/wid/e/eid/variables'
  element = OnshapeElement(url)
  method = 'POST'

  params = {}
  headers = {'Accept': 'application/vnd.onshape.v2+json; charset=UTF-8;qs=0.1',
            'Content-Type': 'application/json'}

  fixed_url = fixed_url.replace('did', element.did)
  fixed_url = fixed_url.replace('wid', element.wvmid)
  fixed_url = fixed_url.replace('eid', element.eid)
  vars = getVariables(url)

  for i,x in enumerate(vars[0]['variables']):
    del vars[0]['variables'][i]['value']
    if varName in x['name']:
      vars[0]['variables'][i]['expression'] = varValue

  payload = vars[0]['variables']

  response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body=payload)

newVars = setVariable(url, varName, varValue)
if showResponse:
  print(json.dumps(newVars, indent=4, sort_keys=True))
else:
  pass


null


### Main loop for updating the variables to count down for the specified time

In [5]:
## import the time module
import time

## define the countdown func.
def countdown(timerIn):
  ## initiate the timer
  t = int(timerIn)
  originalTime = int(timerIn)
  while t > 0:
    mins, secs = divmod(t, 60)
    timer = '{:02d}:{:02d}'.format(mins, secs)
    print(timer)
    start = time.time()
    setVariable(url, "timeRemaining", "\"" + timer + "\"" )
    hourGlassTime = ((originalTime - t)/originalTime)*3.9
    print(str(hourGlassTime))
    setVariable(url, "hourGlassTime", str(hourGlassTime)+"*cm")
    end = time.time()
    callTime = start-end
    print(callTime)
    time.sleep(10)
    timerIn -= 10+callTime
    t = int(timerIn)
    
  print('Right now!')
  setVariable(url, "timeRemaining", "\"Right now!\"")
  setVariable(url, "hourGlassTime", "4*cm")


# input time in seconds
timerIn = input("Enter the time in seconds: ")

# function call
countdown(float(timerIn))


Enter the time in seconds: 100
01:40
0.0
-0.875551700592041
01:30
0.39
-0.6237497329711914
01:21
0.741
-0.6287891864776611
01:12
1.092
-0.5999963283538818
01:02
1.482
-0.5994679927825928
00:53
1.833
-0.5913779735565186
00:43
2.223
-0.6082549095153809


KeyboardInterrupt: ignored